<a href="https://colab.research.google.com/github/Jeli04/ChatBot/blob/main/Chatbot_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np 
import pandas as pd 
import csv
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D



data = pd.read_csv("/content/data/20200325_counsel_chat.csv", encoding='utf-8')
data.head()


,Unnamed: 0,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views,split
0,0,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Sherry Katz, LCSWCouples and Family Therapist,...",https://counselchat.com/therapists/sherry-katz...,"If everyone thinks you're worthless, then mayb...",1,2899,train
1,1,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Robin Landwehr, DBH, LPCC, NCCMental Health in...",https://counselchat.com/therapists/robin-landw...,"Hello, and thank you for your question and see...",1,3514,train
2,2,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,Lee KingI use an integrative approach to treat...,https://counselchat.com/therapists/lee-king,First thing I'd suggest is getting the sleep y...,0,5,train
3,3,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Shauntai Davis-YearginPersonalized, private on...",https://counselchat.com/therapists/shauntai-da...,Therapy is essential for those that are feelin...,0,31,train
4,4,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,Jordan WhiteLicensed Social Worker at Oak Root...,https://counselchat.com/therapists/jordan-white,I first want to let you know that you are not ...,0,620,train


In [2]:
# open the file in read mode
filename = open('/content/data/20200325_counsel_chat.csv', 'r')
 
# creating dictreader object
file = csv.DictReader(filename)

questions = []
answer_labels = []

for col in file:
  questions.append(col["questionText"])
  answer_labels.append(col["answerText"])

print(len(questions))
print(len(answer_labels))

2129
2129


In [3]:
for i in range(10):
  print("Question: " + questions[i])
  print("Answer: " + answer_labels[i])

Question: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?
Answer: If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media.  Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is somehow terrible.Bad 

**Add the intents**

In [4]:
#intents


In [5]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(answer_labels)
training_labels = lbl_encoder.transform(answer_labels)

num_classes = len(answer_labels)

In [6]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(questions)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(questions)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [7]:
print(word_index)

{'<OOV>': 1, 'i': 2, 'and': 3, 'to': 4, 'a': 5, 'my': 6, 'have': 7, 'me': 8, 'the': 9, 'he': 10, 'of': 11, 'in': 12, 'with': 13, 'is': 14, 'but': 15, 'it': 16, 'that': 17, 'for': 18, 'this': 19, 'do': 20, "i'm": 21, 'how': 22, 'she': 23, 'about': 24, 'feel': 25, 'her': 26, 'been': 27, 'like': 28, 'was': 29, 'him': 30, 'am': 31, "don't": 32, 'know': 33, 'on': 34, 'be': 35, 'so': 36, 'or': 37, 'not': 38, 'get': 39, 'what': 40, 'we': 41, 'just': 42, 'can': 43, 'out': 44, 'when': 45, 'has': 46, 'because': 47, 'want': 48, 'years': 49, 'had': 50, 'never': 51, 'i’m': 52, "i've": 53, 'time': 54, 'if': 55, 'at': 56, 'myself': 57, 'counseling': 58, 'from': 59, 'now': 60, 'they': 61, 'always': 62, 'all': 63, 'his': 64, 'really': 65, 'should': 66, 'issues': 67, 'are': 68, 'relationship': 69, 'anxiety': 70, 'up': 71, 'people': 72, 'still': 73, 'as': 74, 'no': 75, 'think': 76, "can't": 77, 'things': 78, 'many': 79, "it's": 80, 'love': 81, 'them': 82, 'depression': 83, 'very': 84, 'i’ve': 85, 'even':

In [8]:
model = tf.keras.Sequential([Embedding(vocab_size, embedding_dim, input_length=max_len),
                             GlobalAveragePooling1D(), 
                             Dense(16, activation = "relu"),
                             Dense(16, activation = "relu"),
                             Dense(num_classes, activation='softmax')
                             
])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 2129)              36193     
                                                                 
Total params: 52,737
Trainable params: 52,737
Non-trainable params: 0
____________________________________________________

In [10]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(padded_sequences, np.array(training_labels), epochs=500)

Epoch 1/500
67/67 [==============================] - 1s 5ms/step - loss: 7.6740 - accuracy: 4.6970e-04
Epoch 2/500
67/67 [==============================] - 0s 5ms/step - loss: 7.6615 - accuracy: 0.0000e+00
Epoch 3/500
67/67 [==============================] - 0s 5ms/step - loss: 7.6490 - accuracy: 9.3941e-04
Epoch 4/500
67/67 [==============================] - 0s 5ms/step - loss: 7.5237 - accuracy: 0.0019
Epoch 5/500
67/67 [==============================] - 0s 5ms/step - loss: 7.2171 - accuracy: 0.0014
Epoch 6/500
67/67 [==============================] - 0s 5ms/step - loss: 6.8598 - accuracy: 0.0019
Epoch 7/500
67/67 [==============================] - 0s 5ms/step - loss: 6.5250 - accuracy: 0.0019
Epoch 8/500
67/67 [==============================] - 0s 5ms/step - loss: 6.2374 - accuracy: 0.0038
Epoch 9/500
67/67 [==============================] - 0s 5ms/step - loss: 5.9806 - accuracy: 0.0052
Epoch 10/500
67/67 [==============================] - 0s 5ms/step - loss: 5.7424 - accuracy: 0.00

In [11]:
model.save("chat_model")

INFO:tensorflow:Assets written to: chat_model/assets


In [17]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /gdrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[Errno 2] No such file or directory: '/gdrive'
/content
